In [1]:
%cd ../airflow
%pwd

/home/welberthime/Development/fiap/cartolafc/airflow


'/home/welberthime/Development/fiap/cartolafc/airflow'

In [2]:
from include import RawExtractor

extractor = RawExtractor(
    base_url="https://api.github.com/repos/henriquepgomide/caRtola/contents/data",
    path="./data/raw",
)

extractor.extract_static_files("2017")
extractor.extract_dynamic_files("2017")

In [17]:
import pandas as pd
import uuid

In [26]:
clubes_df = pd.read_csv("./data/raw/2017/times_ids/1.csv")
clubes_df = clubes_df[["id", "cod.2017", "nome.cbf"]].rename(
    columns={"cod.2017": "olderID", "id": "clubeID", "nome.cbf": "nome"}
)

partidas_df = pd.read_csv("./data/raw/2017/partidas/1.csv")

partidas_df = partidas_df.merge(
    right=clubes_df,
    left_on="home_team",
    right_on="nome",
    suffixes=(None, "_mandante"),
)

partidas_df = partidas_df.merge(
    right=clubes_df,
    left_on="away_team",
    right_on="nome",
    suffixes=(None, "_visitante"),
)

partidas_df["mandantePlacar"] = partidas_df["score"].str.split(" x ").str[0].astype(int)
partidas_df["visitantePlacar"] = partidas_df["score"].str.split(" x ").str[-1].astype(int)

def get_resultado(row):
    if row["mandantePlacar"] > row["visitantePlacar"]:
        return "Casa"
    elif row["mandantePlacar"] < row["visitantePlacar"]:
        return "Visitante"
    else:
        return "Empate"
    
partidas_df["resultado"] = partidas_df.apply(get_resultado, axis=1)
partidas_df["partidaID"] = [uuid.uuid4().hex for _ in range(len(partidas_df))]
    
partidas_df["temporada"] = 2017
partidas_df = partidas_df.rename(
    columns={
        "clubeID": "clubeMandanteID",
        "clubeID_visitante": "clubeVisitanteID",
        "round": "rodada",
    }
)

partidas_df = partidas_df[[
    "partidaID",
    "clubeMandanteID",
    "clubeVisitanteID",
    "mandantePlacar",
    "visitantePlacar",
    "rodada",
    "resultado",
    "temporada",
]]

partidas_df

,partidaID,clubeMandanteID,clubeVisitanteID,mandantePlacar,visitantePlacar,rodada,resultado,temporada
0,beb47dd7a21047dc8b49e0f1c2f3d663,266,277,3,2,1,Casa,2017
1,3639d9e081b64bb698343deb1c84af3e,262,277,1,2,37,Visitante,2017
2,b5e5a5ab76e44f99b257fec0ef485cf4,275,277,0,1,26,Visitante,2017
3,d95ff71cd19b4de2925d1f6629c8d692,264,277,2,0,4,Casa,2017
4,07f3119322c04508b40e3a74beb6216c,283,277,1,1,22,Empate,2017
...,...,...,...,...,...,...,...,...
375,5257246c338546a38ab3d96cd56d7b99,293,266,3,1,24,Casa,2017
376,b7e266280acd4451a081c5c4e77263e7,292,266,2,2,18,Empate,2017
377,130f063fde1f4212847c584089e9079d,287,266,2,2,23,Empate,2017
378,90b3082c24f741afb50230d59839eb19,373,266,1,1,38,Empate,2017


In [31]:
clubes_df = pd.read_csv("./data/raw/2017/times_ids/1.csv")
clubes_df = clubes_df[["id", "cod.2017", "nome.cartola"]].rename(
    columns={"cod.2017": "olderID", "id": "clubeID", "nome.cartola": "nome"}
)

scouts_df = pd.read_csv("./data/raw/2017/scouts_raw/1.csv")

scouts_df = scouts_df.merge(
    right=clubes_df,
    left_on="atletas.clube.id.full.name",
    right_on="nome",
)

scouts_df = scouts_df.rename(
    columns={
        'atletas.atleta_id': "atletaID",
        'Rodada': "rodada",
        'atletas.pontos_num': "pontos",
        'atletas.media_num': "pontosMedia",
        'atletas.preco_num': "preco", 
        'atletas.variacao_num': "precoVariacao",
    },
)

house_by_rounds = partidas_df[["partidaID", "rodada", "clubeMandanteID"]].rename(
    columns={"clubeMandanteID": "clubeID"}
)
visitor_by_rounds = partidas_df[["partidaID", "rodada", "clubeVisitanteID"]].rename(
    columns={"clubeVisitanteID": "clubeID"}
)
club_by_rounds = pd.concat([house_by_rounds, visitor_by_rounds])

scouts_df = scouts_df.merge(
    right=club_by_rounds,
    on=["rodada", "clubeID"],
)

scouts_df = scouts_df[[
    "clubeID",
    "partidaID",
    "atletaID",
    "pontos",
    "pontosMedia",
    "preco",
    "precoVariacao",
    "FS",
    "PE",
    "A",
    "FT",
    "FD",
    "FF",
    "G",
    "I",
    "PP",
    "RB",
    "FC",
    "GC",
    "CA",
    "CV",
    "SG",
    "DD",
    "DP",
    "GS",
]]

scouts_df

,clubeID,partidaID,atletaID,pontos,pontosMedia,preco,precoVariacao,FS,PE,A,...,PP,RB,FC,GC,CA,CV,SG,DD,DP,GS
0,262.0,85fa85095c81474ea2bb0308dc869cc9,36540,0.0,0.00,5.00,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,262.0,85fa85095c81474ea2bb0308dc869cc9,38311,6.2,6.20,8.63,2.63,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,262.0,85fa85095c81474ea2bb0308dc869cc9,38909,0.0,0.00,25.00,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,262.0,85fa85095c81474ea2bb0308dc869cc9,50301,7.2,7.20,12.87,1.87,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,262.0,85fa85095c81474ea2bb0308dc869cc9,50348,0.0,0.00,12.00,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33199,373.0,90b3082c24f741afb50230d59839eb19,93551,1.7,2.56,4.59,-0.84,45.0,57.0,2.0,...,NaN,46.0,72.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN
33200,373.0,90b3082c24f741afb50230d59839eb19,95438,0.0,0.00,1.00,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33201,373.0,90b3082c24f741afb50230d59839eb19,95550,0.0,0.00,0.79,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33202,373.0,90b3082c24f741afb50230d59839eb19,93956,0.0,0.00,2.00,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
clubes_df = pd.read_csv("./data/raw/2017/times_ids/1.csv")
clubes_df = clubes_df[["id", "cod.2017"]].rename(
    columns={"cod.2017": "olderID", "id": "clubeID"}
).drop_duplicates()

atletas_df = pd.read_csv("./data/raw/2017/jogadores/1.csv")

atletas_df = atletas_df.merge(
    right=clubes_df,
    left_on="ClubeID",
    right_on="olderID",
)

atletas_df.drop_duplicates(inplace=True)
atletas_df.rename(
    columns={
        "AtletaID": "atletaID",
        "Apelido": "apelido",
        "PosicaoID": "posicaoID",
    },
    inplace=True
)

atletas_df = atletas_df[[
    "atletaID",
    "clubeID",
    "posicaoID",
    "apelido",
]]

atletas_df

,atletaID,clubeID,posicaoID,apelido
0,36540,262,3,Juan
1,38311,262,4,Márcio Araújo
2,38509,262,1,Diego Alves
3,38909,262,4,Diego
4,42501,262,3,Rhodolfo
...,...,...,...,...
746,91917,373,4,Silva
747,92981,373,4,Ronaldo
748,93551,373,4,Igor
749,93806,373,5,Alison


In [35]:
posicoes_df = pd.read_csv("./data/raw/2017/posicoes_ids/1.csv")
posicoes_df.rename(
    columns={
        "Cod": "posicaoID",
        "Position": "nome",
        "abbr": "abreviacao",
    },
    inplace=True,
)

posicoes_df = posicoes_df[[
    "posicaoID",
    "nome",
    "abreviacao",
]]

posicoes_df

,posicaoID,nome,abreviacao
0,1,Goleiro,gol
1,2,Lateral,lat
2,3,Zagueiro,zag
3,4,Meia,mei
4,5,Atacante,ata
5,6,Técnico,tec


In [36]:
clubes_df = pd.read_csv("./data/raw/2017/times_ids/1.csv")
clubes_df.rename(
    columns={
        "id": "clubeID",
        "nome.cbf": "nome",
    },
    inplace=True,
)

clubes_df = clubes_df[["clubeID", "nome", "abreviacao"]]
clubes_df.drop_duplicates(subset=["clubeID"], keep="last", inplace=True)

clubes_df

,clubeID,nome,abreviacao
0,327,América - MG,AME
1,200,America - RN,OUT
2,373,Atlético - GO,ATL
3,282,Atlético - MG,ATL
6,293,Athlético-PR,ATL
7,314,Avaí - SC,AVA
8,265,Bahia - BA,BAH
9,263,Botafogo - RJ,BOT
10,203,Brasiliense,BRA
13,354,Ceará - CE,CEA
